<br><H3>Feature: Building Web Scraper to get related news blogs, articles related to User-input keywords</H3></br>
<br></br>
Author: Ronak Dedhia
<br></br>
Version: 1.1
<br></br>
Date: 22 September, 2018
<br></br>
<H5>Revision 1:</H5>
Revision Date: 23 September, 2018
<br></br>
Take multiple keywords from user and display only top 5 articles of each keyword
<br></br>
<H5>Revision 2:</H5>
Revision Date: 28 September, 2018
<br></br>
Introduce exception handling to limit the number of keywords and notifications if no articles found, invalid number


In [1]:
#Installing the required packages
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install datetime
!{sys.executable} -m pip install bs4


In [2]:

#Importing required Packages
import pandas as pd
from collections import OrderedDict
from datetime import datetime
import urllib
import requests
from bs4 import BeautifulSoup

#Set all lists to capture different elements of the search
kywd = []
sr_no = []
news_title = []
news_dt = []
news_link = []
news_kywd = []
raw_keyword = []  # For Displaying the topic

# Get user input of keywords for which they want to recieve alerts
while True:
    try:
        kywd_cnt = int(input('How many key words do you want(Maximum 5 keywords)? '))
        if(kywd_cnt < 6 and kywd_cnt != 0):
            break
        print('Enter atleast 1 or upto 5 keywords')
    except ValueError:
        print("No valid integer! Please try again ...")

for i in range(kywd_cnt):
    # 1.Take input  2. Quote it to fit the url format 3. Append it to keywd variable
    raw_keyword.append(input('Enter keyword {}:'.format(i+1)))
    kywd.append(urllib.parse.quote(raw_keyword[i]))
    
for x in range(len(kywd)):
    #Request seperate web page for results based on different keywords    
    page = requests.get("http://www.denofgeek.com/search/site/{}".format(kywd[x]))
    page_status_code = page.status_code                   #Value 200 indicates page downloaded successfully
    soup = BeautifulSoup(page.content, 'html.parser')     #Create instance of Beautiful Soup
    #Filter all the tags containing the search articles
    outer_list = soup.find_all('li', class_= 'search-result clearfix')
    
    #Check and notify if no articles exist with the keywords
    if not outer_list:
        print('No results found for the keyword "{}"'.format(raw_keyword[x]))
        
    #Loop through the tags inside lists to get required data
    for article in enumerate(outer_list):
        dt = article[1].find('div', class_='date')
        link = article[1].div.find('a', href=True)
        if link is not None:
            sr_no.append(article[0] + 1)
            news_title.append(article[1].h2.text.strip())
            news_link.append(link['href'].strip())
            news_dt.append(dt.text.strip())
            news_kywd.append(raw_keyword[x])

#Defining the dataframe for storing topics
news_table = {
            #'sr_no':sr_no,
            'news_title':news_title,
            'news_date':news_dt,
             'news_link': news_link,
            'news_keyword': news_kywd}
news_table['news_date_format'] = pd.to_datetime(news_table['news_date'], format='%b %d, %Y', errors='ignore')
news_table = pd.DataFrame(OrderedDict(news_table))

#Sort the values based on datetime and topic
news_table.sort_values(by=['news_keyword', 'news_date_format'],ascending= False, inplace= True)

#Drop the datetime column after sorting is done
news_table.drop('news_date_format', axis=1, inplace= True)

#Display the pretty version of the extracted data (Top 5 or less depending on the topic)
pd.set_option('display.max_colwidth', -1) #Show entire text for article links

news_table.groupby('news_keyword').head(5).reset_index(drop=True)

How many key words do you want(Maximum 5 keywords)? 8
Enter atleast 1 or upto 5 keywords
How many key words do you want(Maximum 5 keywords)? @#
No valid integer! Please try again ...
How many key words do you want(Maximum 5 keywords)? 3
Enter keyword 1:Venom
Enter keyword 2:Tom Hardy
Enter keyword 3:SmallFoot
No results found for the keyword "SmallFoot"


,news_date,news_link,news_keyword,news_title
0,"Oct 12, 2018",http://www.denofgeek.com/uk/movies/2018-movies/53716/40-films-2018,Venom,40 essential films to check out in 2018
1,"Oct 5, 2018",http://www.denofgeek.com/uk/movies/60907/which-scenes-were-cut-from-venom,Venom,Which scenes were cut from Venom?
2,"Oct 3, 2018",http://www.denofgeek.com/uk/movies/60873/venom-review-a-silly-mess-that-tom-hardy-cant-save,Venom,"Venom review: a silly mess, that Tom Hardy can't save"
3,"Oct 2, 2018",http://www.denofgeek.com/uk/movies/60847/venom-first-reactions-roundup,Venom,Venom: first reactions roundup
4,"Sep 24, 2018",http://www.denofgeek.com/uk/movies/venom/60646/riz-ahmed-on-venom-its-more-like-a-werewolf-movie-than-a-superhero-movie,Venom,"Riz Ahmed on Venom: ""it's more like a werewolf movie than a superhero movie"""
5,"Oct 5, 2018",http://www.denofgeek.com/uk/movies/60907/which-scenes-were-cut-from-venom,Tom Hardy,Which scenes were cut from Venom?
6,"Oct 3, 2018",http://www.denofgeek.com/uk/movies/60873/venom-review-a-silly-mess-that-tom-hardy-cant-save,Tom Hardy,"Venom review: a silly mess, that Tom Hardy can't save"
7,"Sep 19, 2018",http://www.denofgeek.com/uk/movies/venom/60450/venom-heading-for-15-certificate-in-the-uk-new-poster,Tom Hardy,"Venom: heading for 15 certificate in the UK, new poster"
8,"Aug 9, 2018",http://www.denofgeek.com/uk/movies/fonzo/59556/tom-hardy-unrecognisable-as-al-capone,Tom Hardy,Tom Hardy gets ugly as old man Al Capone
9,"Jul 31, 2018",http://www.denofgeek.com/uk/movies/venom/55153/venom-heres-the-second-new-trailer-full-of-hardy,Tom Hardy,"Venom: here's the second new trailer, full of Hardy"
